In [1]:
# !pip install langchain

Defaulting to user installation because normal site-packages is not writeable


In [4]:
import os
import openai
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI


from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())
openai.api_key = os.getenv("OPENAI_API_KEY")


In [5]:
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage,
)


chat = ChatOpenAI(model='gpt-3.5-turbo')

In [6]:
messages = [
    SystemMessage(content="You only answer in german."),
    HumanMessage(content="Tell me a joke."),
]

chat(messages)

AIMessage(content='Was macht ein Clown im Büro?\n\nFaxen!', additional_kwargs={}, example=False)

In [7]:
from langchain import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

template="""Interprete the text and evaluate the text.
sentiment: is the text positive, negative or neutral?
subject: What is the subject of the text? Use exactly one word not two or more.

Format the output as JSON with the following keys:
sentiment
subject
"""
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [8]:
# using the ChatPromptTemplate to combine the two prompts
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

chat(chat_prompt.format_prompt(text="I ordered a kitchen table, but it was scratched").to_messages())

AIMessage(content='{\n  "sentiment": "negative",\n  "subject": "product"\n}', additional_kwargs={}, example=False)

In [9]:
# using a chain to run the prompt
chain = LLMChain(llm=chat, prompt=chat_prompt)
chain.run(text="I ordered a kitchen table, but it was scratched")

'{\n  "sentiment": "negative",\n  "subject": "kitchen table"\n}'

# Real World example with ResponseSchema, Templates, Chains and OutputParsers



In [10]:
from langchain.output_parsers import (
    ResponseSchema,
    StructuredOutputParser
)

sentiment_schema = ResponseSchema(name="sentiment", description="Is the text positive, negative or neutral? Only use one of the provided words.")
subject_schema = ResponseSchema(name="subject", description="What is the subject of the text? Use exactly one word.")
price_schema = ResponseSchema(name="price", description="What is the price of the product? Use none if no price was provided in the text.")

response_schemas = [sentiment_schema, subject_schema, price_schema]

In [11]:
parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()
format_instructions

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"sentiment": string  // Is the text positive, negative or neutral? Only use one of the provided words.\n\t"subject": string  // What is the subject of the text? Use exactly one word.\n\t"price": string  // What is the price of the product? Use none if no price was provided in the text.\n}\n```'

In [12]:
template="""Interprete the text and evaluate the text.
sentiment: is the text positive, negative or neutral?
subject: What is the subject of the text? Use exactly one word not two or more.

text: {input}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=template)
format_instructions = parser.get_format_instructions()

messages = prompt.format_messages(input="I ordered a kitchen table, but it was scratched", format_instructions=format_instructions)
chat = ChatOpenAI(temperature=0.0)
response = chat(messages)
response

AIMessage(content='```json\n{\n\t"sentiment": "negative",\n\t"subject": "table",\n\t"price": "none"\n}\n```', additional_kwargs={}, example=False)

In [13]:
output_dict = parser.parse(response.content)
output_dict

{'sentiment': 'negative', 'subject': 'table', 'price': 'none'}

# Using Sequential Chains to Pass Output from One Model to Another

In [14]:
template="""Interprete the text and evaluate the text.
sentiment: is the text positive, negative or neutral?
subject: What is the subject of the text? Use exactly one word not two or more.

Format the output as JSON with the following keys:
sentiment
subject
"""
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chain = LLMChain(llm=chat, prompt=chat_prompt)

response_template="""
You are a helpful bot that creates a "Thank you" message for a customer.
If the customer is unsatisfied, offer them a real world assistant to talk to.
You will get a sentiment and subject as input.

text: {input}
"""

review_template=PromptTemplate(input_variables=["input"], template=response_template)
review_chain = LLMChain(llm=chat, prompt=review_template)





In [15]:
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(chains=[chain, review_chain], verbose=True)

overall_chain.run(input="I ordered a kitchen table, but it was scratched")



> Entering new SimpleSequentialChain chain...
{
  "sentiment": "negative",
  "subject": "kitchen table"
}
Dear valued customer,

Thank you for reaching out to us regarding your kitchen table. We appreciate your feedback, although we are sorry to hear that you are unsatisfied with your purchase. 

We understand that sometimes it can be frustrating when things don't meet your expectations. In order to provide you with the best possible assistance, we would like to offer you the opportunity to speak with one of our real-world assistants who can better address your concerns and find a suitable solution for you. They will be able to provide you with personalized assistance and guide you through the process of resolving any issues you may have with your kitchen table.

Please let us know if you would like to take advantage of this offer by replying to this message. We are committed to ensuring your satisfaction and will do everything we can to assist you.

Thank you for your understanding 

"Dear valued customer,\n\nThank you for reaching out to us regarding your kitchen table. We appreciate your feedback, although we are sorry to hear that you are unsatisfied with your purchase. \n\nWe understand that sometimes it can be frustrating when things don't meet your expectations. In order to provide you with the best possible assistance, we would like to offer you the opportunity to speak with one of our real-world assistants who can better address your concerns and find a suitable solution for you. They will be able to provide you with personalized assistance and guide you through the process of resolving any issues you may have with your kitchen table.\n\nPlease let us know if you would like to take advantage of this offer by replying to this message. We are committed to ensuring your satisfaction and will do everything we can to assist you.\n\nThank you for your understanding and patience.\n\nBest regards,\n[Your Name]"